# Circlaridade medica Rede Neural e Random Forest

In [ ]:
import pandas as pd

#Base que sera usada na classificação
datasetEstudantes = pd.read_csv('base_caract.csv', sep=',')
#Base que mantera todas as colunas para fazer as analises
datasetMedicos = pd.read_csv('base_medicos.csv', sep=',')
# 'co_uf_ies', 'dist', 'idh_uf_ies', 'dt_inicio_curso', 'lat_nasc', 'lon_nasc', 'lat_ies', 'lon_ies'

In [ ]:
datasetEstudantes.head()

In [ ]:
datasetMedicos.head()

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans

# Carregando o dataset
datasetEstudantes = pd.read_csv('base_caract.csv', sep=',')

# K-means clustering para discretização
kmeans = KMeans(n_clusters=10, random_state=42)
datasetEstudantes['idade_bin'] = kmeans.fit_predict(datasetEstudantes[['idade']])

# Mapeamento das idades originais para as idades discretizadas
idade_mapping = datasetEstudantes[['idade', 'idade_bin']].drop_duplicates(subset='idade').sort_values(by='idade')

print("Mapeamento das idades originais para as idades discretizadas:")
print(idade_mapping)


In [ ]:
import matplotlib.pyplot as plt

# Contar as instâncias de cada classe
class_counts = datasetEstudantes['co_uf_ies'].value_counts()

# Visualização com gráfico de barras
class_counts.plot(kind='bar')
plt.xlabel('Classes')
plt.ylabel('Número de Instâncias')
plt.title('Distribuição das Classes')
plt.savefig('distribuicaoClasses.png')
plt.show()

# Calcular e mostrar a proporção de cada classe
class_proportions = class_counts / class_counts.sum()
print("Proporção de cada classe:")
print(class_proportions)


In [ ]:
datasetMedicos.head()

In [ ]:
import csv
import numpy as np
import category_encoders as ce
from datetime import datetime

#Base que sera usada na classificação
dataset = pd.read_csv('base_caract.csv', sep=',')
#Base que mantera todas as colunas para fazer as analises
dataset_aux = pd.read_csv('base_caract.csv', sep=',')

dataset['idade'] = datasetEstudantes['idade_bin']

#codificando os codigos dos estados
data = dataset['co_uf_ies']
encoder_ies=ce.OneHotEncoder(cols=['co_uf_ies'])
classes = encoder_ies.fit_transform(data).values

dataset.pop('co_uf_ies')
dist = dataset.pop('dist')
idh_uf_ies = dataset.pop('idh_uf_ies')
dt_inicio_curso = dataset.pop('dt_inicio_curso')
lat_nasc = dataset.pop('lat_nasc')
lon_nasc = dataset.pop('lon_nasc')
lat_ies = dataset.pop('lat_ies')
lon_ies = dataset.pop('lon_ies')
idh_uf_nasc = dataset.pop('idh_uf_nasc')

co_uf_nasc = dataset['co_uf_nasc']
encoder=ce.OneHotEncoder(cols=['co_uf_nasc'])
#dataset = encoder.fit_transform(dataset).values

In [ ]:
dataset = encoder.fit_transform(dataset).values

In [ ]:
# cria o arquivo
open_doc = open('teste.csv', 'w', newline='', encoding='utf-8')
# cria o objeto de gravação
w = csv.writer(open_doc)
w.writerow(["cpf", "idade", "sexo", "raça",
            "co_uf_nasc_enc", "co_uf_nasc_enc",
            "co_uf_nasc_enc", "co_uf_nasc_enc",
            "co_uf_nasc_enc", "co_uf_nasc_enc",
            "co_uf_nasc_enc", "co_uf_nasc_enc",
            "co_uf_nasc_enc", "co_uf_nasc_enc",
            "co_uf_nasc_enc", "co_uf_nasc_enc",
            "co_uf_nasc_enc", "co_uf_nasc_enc",
            "co_uf_nasc_enc", "co_uf_nasc_enc",
            "co_uf_nasc_enc", "co_uf_nasc_enc",
            "co_uf_nasc_enc", "co_uf_nasc_enc",
            "co_uf_nasc_enc", "co_uf_nasc_enc",
            "co_uf_nasc_enc", "co_uf_nasc_enc",
            "co_uf_nasc_enc", "co_uf_nasc_enc",
            "co_uf_nasc_enc", "cod_uf"])

for l in dataset[:5]:
    w.writerow(list(map(str, l)))
dataset_teste = pd.read_csv('teste.csv', sep=',')
dataset_teste.head()

In [ ]:
dataset_teste.iloc[:, [*range(10)] + [-2, -1]]

Rede neural com K-Fold e classificaation report

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import KFold
from category_encoders import OneHotEncoder

# Base que será usada na classificação
dataset = pd.read_csv('base_caract.csv', sep=',')
# Base que manterá todas as colunas para fazer as análises
dataset_aux = pd.read_csv('base_caract.csv', sep=',')

# dataset['idade'] = datasetEstudantes['idade_bin']

# Codificando os códigos dos estados para rótulos
encoder_ies = OneHotEncoder(cols=['co_uf_ies'])
classes = encoder_ies.fit_transform(dataset[['co_uf_ies']])

# Removendo colunas que não serão usadas
dataset.drop(columns=['cpf', 'idade', 'sexo', 'raca', 'dt_inicio_curso'], inplace=True)

# Codificando as características dos estados de nascimento
encoder = OneHotEncoder(cols=['co_uf_nasc'])
dataset_encoded = encoder.fit_transform(dataset)

# Convertendo para numpy array
X = dataset_encoded.values
y = classes.values.argmax(axis=1)  # Convertendo one-hot encoded labels para índices

# Verificando se os dados foram processados corretamente
print("Shape de X:", X.shape)
print("Shape de y:", y.shape)
print("Primeiros 5 rótulos:", y[:5])

# Normalizando os dados
X = (X - X.mean(axis=0)) / X.std(axis=0)

# Obtendo o número de classes
num_classes = classes.shape[1]

# K-Fold Cross Validation
kf = KFold(n_splits=5)
best_accuracy = 0
best_fold = None
fold_index = 0

for train_index, test_index in kf.split(X):
    fold_index += 1
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Convertendo os dados para tensores
    X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
    y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.int32)
    X_test_tensor = tf.convert_to_tensor(X_test, dtype=tf.float32)
    y_test_tensor = tf.convert_to_tensor(y_test, dtype=tf.int32)

    # Construindo o modelo da rede neural
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(128, activation='relu'),  # Camada oculta adicional 1
        tf.keras.layers.Dense(256, activation='relu'),  # Camada oculta adicional 2
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

    # Compilando o modelo
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Treinando o modelo
    model.fit(X_train_tensor, y_train_tensor, epochs=10, batch_size=32, validation_split=0.2, verbose=2)

    # Avaliando o desempenho no conjunto de teste
    y_pred_probs = model.predict(X_test_tensor)
    y_pred = tf.argmax(y_pred_probs, axis=1).numpy()  # Convertendo para array NumPy

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    report = classification_report(y_test, y_pred, target_names=[f'class_{i}' for i in range(num_classes)])

    train_percentage = len(train_index) / len(X) * 100
    test_percentage = len(test_index) / len(X) * 100

    print(f'Fold {fold_index}:')
    print('Acurácia: {:.2f}%'.format(accuracy * 100))
    print('Precisão: {:.2f}%'.format(precision * 100))
    print('Recall: {:.2f}%'.format(recall * 100))
    print('F1-Score: {:.2f}%'.format(f1 * 100))
    print('Porcentagem Treino: {:.2f}%'.format(train_percentage))
    print('Porcentagem Teste: {:.2f}%'.format(test_percentage))
    print('Relatório de Classificação:')
    print(report)
    print()

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_fold = (train_index, test_index)

print('Melhor Fold:')
print(f'Índices de Treino: {best_fold[0]}')
print(f'Índices de Teste: {best_fold[1]}')
print('Melhor Acurácia: {:.2f}%'.format(best_accuracy * 100))


Random Forest com Grid Search, K Cross Validation e Classsification report

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.ensemble import RandomForestClassifier

# Carregar o conjunto de dados
dataset = pd.read_csv('base_caract.csv', sep=',')

# Mapear as idades
dataset['idade'] = datasetEstudantes['idade_bin']

# Remover colunas não utilizadas
dataset.drop(columns=['cpf', 'idade', 'sexo', 'raca', 'dt_inicio_curso'], inplace=True)

# Dividir o conjunto de dados em features e classes
X = dataset.drop(columns=['co_uf_ies']).values  # Features (tudo exceto a coluna alvo)
y = dataset['co_uf_ies'].values  # Target (a coluna que queremos prever)

# Configurar os parâmetros do K-Fold Cross Validation
num_splits = 5
kf = KFold(n_splits=num_splits, shuffle=True, random_state=42)

# Definir os parâmetros para o Grid Search
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30]
}

# Inicializar o modelo Random Forest
model_rand = RandomForestClassifier()

# Inicializar o Grid Search
grid_search = GridSearchCV(estimator=model_rand, param_grid=param_grid, cv=kf, scoring='accuracy')

# Executar o Grid Search
grid_search.fit(X, y)

# Imprimir os melhores parâmetros encontrados
print("Melhores Parâmetros:")
print(grid_search.best_params_)

# Imprimir a melhor acurácia encontrada
print("Melhor Acurácia: {:.2f}%".format(grid_search.best_score_ * 100))


In [ ]:
plt.scatter(dist, dataset_aux['idade'], label=dataset_aux['idade'])
plt.xlabel('Distancia')
plt.ylabel('Idade')
plt.title('Relação entre idade e distância')
plt.figure(figsize=(20,10))
plt.show

In [ ]:
import folium
from folium.plugins import HeatMap
from datetime import date
import pandas as pd

# Ler os dados
inep_df = pd.read_csv('inep_clean.csv', index_col=0)
inep_df['dt_inicio_curso'] = pd.to_datetime(inep_df['dt_inicio_curso'])
date_from = pd.Timestamp(date(2014,1,1))
date_to = pd.Timestamp(date(2014,4,1))

# Filtrar os dados para o período desejado
ano_df = inep_df[(inep_df.dt_inicio_curso >= date_from) & (inep_df.dt_inicio_curso <= date_to)].copy()

# Criar um mapa folium
m = folium.Map(location=[-15.788, -47.879], zoom_start=4)

# Adicionar pontos de origem dos alunos como um mapa de calor
heat_data = [[row['lat_ies'], row['lon_ies']] for index, row in ano_df.iterrows()]
HeatMap(heat_data).add_to(m)

# Salvar o mapa como um arquivo HTML
m.save('heatmapIesEstudantes.html')


In [ ]:
import folium
from folium.plugins import HeatMap
from datetime import date
import pandas as pd

# Ler os dados
inep_df = pd.read_csv('inep_clean.csv', index_col=0)
inep_df['dt_inicio_curso'] = pd.to_datetime(inep_df['dt_inicio_curso'])
date_from = pd.Timestamp(date(2008, 1, 1))
date_to = pd.Timestamp(date(2008, 12, 31))

# Filtrar os dados para o período desejado
ano_df = inep_df[(inep_df.dt_inicio_curso >= date_from) & (inep_df.dt_inicio_curso <= date_to)].copy()

# Criar um mapa folium
m = folium.Map(location=[-15.788, -47.879], zoom_start=4)

# Adicionar pontos de origem dos alunos como um mapa de calor
heat_data = [[row['lat_ies'], row['lon_ies']] for index, row in ano_df.iterrows()]
HeatMap(heat_data).add_to(m)

# Salvar o mapa como um arquivo HTML
m.save('heatmapIes2008Estudantes.html')


In [ ]:
import folium
from folium.plugins import HeatMap
from datetime import date
import pandas as pd

# Ler os dados
inep_df = pd.read_csv('inep_clean.csv', index_col=0)
inep_df['dt_inicio_curso'] = pd.to_datetime(inep_df['dt_inicio_curso'])
date_from = pd.Timestamp(date(2014,1,1))
date_to = pd.Timestamp(date(2014,4,1))

# Filtrar os dados para o período desejado
ano_df = inep_df[(inep_df.dt_inicio_curso >= date_from) & (inep_df.dt_inicio_curso <= date_to)].copy()

# Criar um mapa folium
m = folium.Map(location=[-15.788, -47.879], zoom_start=4)

# Adicionar pontos de origem dos alunos como um mapa de calor
heat_data = [[row['lat_ies'], row['lon_ies']] for index, row in ano_df.iterrows()]
HeatMap(heat_data).add_to(m)

# Salvar o mapa como um arquivo HTML
m.save('heatmapIesEstudantes.html')
